In [1]:
import json
import pennylane as qml
import pennylane.numpy as np
import scipy

In [9]:

H = np.array([[ 0.39488016,  0.04722628, -0.17943126, -0.03673282],
        [ 0.04722628,  0.37758558,  0.12997088,  0.17848188],
        [-0.17943126,  0.12997088,  0.53582574,  0.01115543],
        [-0.03673282,  0.17848188,  0.01115543,  0.29170851]])

eigvals, eigvecs = np.linalg.eigh(H)
exp_D=np.eye(len(eigvals))*np.exp(2*np.pi*1j*eigvals)
U=np.dot(eigvecs,np.dot(exp_D,np.conjugate(np.transpose(eigvecs))))
def state_prep(params, wires):

    """
    Implements the state preparation circuit.

    Args:
        - params (np.array(float)): Angles [theta_1, theta_2, theta_3] parametrizing
        the RY rotations in the circuit.
        - wires (list): Labels for the circuit wires.
    Returns:
        - Does not return anything since it is a subcircuit.
    """
    

    qml.RY(params[0],wires=wires[-2])
    qml.RY(params[1],wires=wires[-1])
    qml.CNOT(wires=[wires[-2],wires[-1]])
    qml.RY(params[2],wires=wires[-1])
    

dev = qml.device('lightning.qubit', wires = range(8))
 
@qml.qnode(dev)
def qpe_circuit(params):

    """
    Implements the QPE routine for the Unitary U, including initial state 
    preparation using the state_prep subcircuit.

    Args:
        - params (np.array(float)): Angles [theta_1, theta_2, theta_3] parametrizing
        the RY rotations in the state_prep circuit.
    Returns:
        - np.tensor(float): Computational basis probabilities in the estimation wires.
    """
    

    state_prep(params, [6,7])
    qml.QuantumPhaseEstimation(U,target_wires=[6,7],estimation_wires=[0,1,2,3,4,5])

    return qml.probs(wires=[0,1,2,3,4,5])
    

def compute_statistics(params):

    """
    Computes the phase and its uncertainty by postprocessing the results of the QPE circuit.

    Args:
        - params (np.array(float)): Angles [theta_1, theta_2, theta_3] parametrizing
        the RY rotations in the state_prep circuit.
    Returns:
        - mu (float): The phase calculated as a weighted average.
        - sigma (float): The uncertainty calculated as the standard deviation
        for the phase.
    """
    

    probs=qpe_circuit(params)
    
    num=len(probs)
    mu=0
    for i in range(num):
        mu+=probs[i]*(i/(2**6))

    sigma_sq=0
    for i in range(num):
        sigma_sq+=probs[i]*((i/(2**6)-mu)**2)
    sigma=np.sqrt(sigma_sq)

    return mu, sigma
    



In [11]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    ins = np.array(json.loads(test_case_input))
    outs = list(compute_statistics(ins))
    
    return str(outs)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    assert np.allclose(solution_output, expected_output, atol = 1e-4)

In [12]:
# These are the public test cases
test_cases = [
    ('[1.35889209, -0.6219561, -1.31577162]', '[0.121662, 0.120539]'),
    ('[4.40084815, -0.77288063,  0.6425846]', '[0.303264, 0.043692]')
]

In [ ]:
# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")